In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data_dir = "/content/drive/MyDrive/data"


In [5]:
import os

data_path = '/content/drive/MyDrive/data'

breed_folders = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder))]

print("Dog Breeds Found in 'data' Folder:")
for breed in breed_folders:
    print(f" {breed}")

Dog Breeds Found in 'data' Folder:
 Gordon_setter
 Bernese_mountain_dog
 Norwegian_elkhound
 Bedlington_terrier
 Dandie_Dinmont
 Ibizan_hound


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [8]:
data_dir = '/content/drive/MyDrive/data'
img_size = ( 224,224 )
batch_size = 32

In [12]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 603 images belonging to 6 classes.
Found 149 images belonging to 6 classes.


In [13]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [14]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 442s 23s/step - accuracy: 0.5791 - loss: 1.2639 - val_accuracy: 0.9732 - val_loss: 0.0881
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.9682 - loss: 0.1191 - val_accuracy: 0.9799 - val_loss: 0.0880
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9696 - loss: 0.0819 - val_accuracy: 0.9866 - val_loss: 0.0521
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.9845 - loss: 0.0574 - val_accuracy: 0.9799 - val_loss: 0.0412
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9856 - loss: 0.0576 - val_accuracy: 0.9799 - val_loss: 0.0481
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.9825 - loss: 0.0452 - val_accuracy: 0.9732 - val_loss: 0.0693
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.9861 - loss: 0.0339 - val_accuracy: 0.9799 - val_loss: 0.0703
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.9894 - loss: 0.0378 - val_accuracy: 1.0000 - val_los

In [17]:
model.save("dog_breed_classifier.h5")
print(" Model saved as dog_breed_classifier.h5")

✅ Model saved as dog_breed_classifier.h5


In [18]:
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.9886 - loss: 0.0305
Validation Loss: 0.0362
Validation Accuracy: 0.9866


In [19]:
val_generator.reset()
pred_probs = model.predict(val_generator)
y_pred = np.argmax(pred_probs, axis=1)
y_true = val_generator.classes

class_labels = list(val_generator.class_indices.keys())


5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step


In [22]:
!pip install gradio --quiet

import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load model
model = load_model('dog_breed_classifier.h5')


class_labels = ['Bedlington_terrier', 'Bernese_mountain_dog', 'Dandie_Dinmont',
                'Gordon_setter', 'Ibizan_hound', 'Norwegian_elkhound']


def classify_dog(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    preds = model.predict(img_array)
    return {class_labels[i]: float(preds[0][i]) for i in range(len(class_labels))}

interface = gr.Interface(
    fn=classify_dog,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="Dog Breed Classifier",
    description="Upload a dog image and get the predicted breed!"
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b56d0f0eff6af6adbd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
